In [1]:
import pandas as pd
import os

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

from tkinter import messagebox

import re
import time


In [2]:
print('正在打开……')
df_mr = pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath1 = filedialog.askopenfilename()


正在打开……


### 读取源数据


In [3]:
df1 = pd.read_excel(datapath1, sheet_name='终端购机', dtype='str')
df2 = pd.read_excel(datapath1, sheet_name='我宽异号', dtype='str')
df3 = pd.read_excel(datapath1, sheet_name='全家享2.0', dtype='str')
df4 = pd.read_excel(datapath1, sheet_name='我号异宽', dtype='str')


将中文是和否，替换为 0 和 1


In [79]:
df4['条件'] = df4.apply(lambda x: ((x['高价值80元以上客户'])+(x['办理7折提值大于0客户'] )+(x['办理7折提值30-70客户'] )+(x['终端换机'])+(x['4G资费'] )+(x['预警'])+(x['不限量'])), axis=1)


In [80]:
df1['是否为宽带客户'] = df1['是否为宽带客户'].map({'无宽客户': 0, '有宽客户': 1})



### 备份


In [5]:
df1_bck = df1.copy()
df2_bck = df2.copy()
df3_bck = df3.copy()
df4_bck = df4.copy()


### 还原


In [91]:
df1 = df1_bck
df2 = df2_bck
df3 = df3_bck
df4 = df4_bck


### 读取派单规则


In [7]:
print('正在打开……')
df_mr = pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath_r = filedialog.askopenfilename()


正在打开……


In [8]:
df_r1 = pd.read_excel(datapath_r, sheet_name='渠道')
df_r2 = pd.read_excel(datapath_r, sheet_name='直销')
df_r3 = pd.read_excel(datapath_r, sheet_name='三大家')
df_r4 = pd.read_excel(datapath_r, sheet_name='装维')


### 数据清理


将不是数字类型的数字转化为数字类型。


In [81]:
# df1[['服务中心ID','网格ID']] = df1[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df2[['服务中心ID','网格ID']] = df2[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df3[['服务中心ID','网格ID']] = df3[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# 'raise'参数：无效的解析将引发异常
# 'corece'参数：将无效解析设置为NaN
# 'ignore'参数：无效的解析将返回输入,也就是遇到无效数据不解析，也不会填充NaN或者0
# 用ignore保留中文
df1 = df1.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df2 = df2.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df3 = df3.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df4 = df4.apply(pd.to_numeric, errors='ignore').fillna(0.0)


### 核心函数


递归，根据各种不同条件，取满足条件的号码，并依次遍历，实际上就是将 get 和 set 改变为装饰器，也就是点访问

df\_\_t_tmp 满足条件的号码池
pdl 派单量
row 打标，分配编码
stack 堆栈，存储满足各种条件的号码池
df_res_total 存放本次循环的结果
df_res 取满足派单量的数据
df_orgin 原始 df


In [54]:
# 取出各条件号码，依次遍历

def handle(df_t_tmp,pdl, row, queue, df_res_total, df_res, df_orgin):
    # print(len(df_t_tmp))
    # 结束条件df_end长度
    if (len(df_t_tmp) == 10000):
        return df_res_total
 
        
    if ((len(df_t_tmp) >= pdl)):
        df_res = df_t_tmp.head(pdl).copy()
        df_orgin = df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        return df_res_total
    else:
        last_pdl = pdl - len(df_t_tmp)
        df_res = df_t_tmp.head(len(df_t_tmp)).copy()
        df_orgin = df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        df_t_tmp = queue.pop(0)  # 队列出队
        return handle(df_t_tmp, last_pdl, row, queue, df_res_total, df_res, df_orgin)


转置，用栈 reverse


In [33]:
# 我宽异号
wkyh_term = []

# 全家享
qjx_term = [9]
# 终端购机
zdgj_term = [0, 1]
# 我号异宽
whyk_term = []


In [34]:
def numdec(n, term):
    tmp = []
    for i in range(0, n):
        tmp.append(i)
    for i in range(0, n):
        term.append(tmp.pop(-1))


In [35]:
numdec(32, wkyh_term)
numdec(128, whyk_term)


条件结构体 struct


In [36]:
class wkkh_term(object):

    def __init__(self, res):

        self.res_list = str(res)

        self.res = []

        for v in self.res_list:
            self.res.append(int(v))
    # 我号异宽

    def df_whyk(self, df_tmp):

        df_tmp_res = df_tmp[(df_tmp['高价值80元以上客户'] == self.res[0]) & (df_tmp['办理7折提值大于0客户'] == self.res[1]) & (df_tmp['办理7折提值30-70客户'] == self.res[2]) &
                            (df_tmp['终端换机'] == self.res[3]) & (df_tmp['4G资费'] == self.res[4]) & (df_tmp['预警'] == self.res[5]) & (df_tmp['不限量'] == self.res[6])]
        return df_tmp_res

    # 我宽异号
    def df_wkyh(self, df_tmp):
        df_tmp_res = df_tmp[(df_tmp['我宽异号'] == self.res[0]) & (df_tmp['全家享权益包2.0'] == self.res[1]) & (
            df_tmp['裸奔高价值'] == self.res[2]) & (df_tmp['免费成员空间'] == self.res[3]) & (df_tmp['老旧设备'] == self.res[4])]
        return df_tmp_res

    # 全家享
    def df_qjx(self, df_tmp):
        df_tmp_res = df_tmp[(df_tmp['我宽异号'] == self.res[0]) & (df_tmp['全家享权益包2.0'] == self.res[1]) & (
            df_tmp['裸奔高价值'] == self.res[2]) & (df_tmp['免费成员空间'] == self.res[3]) & (df_tmp['老旧设备'] == self.res[4])]
        return df_tmp_res

    # 终端购机
    def df_zdgj(self, df_tmp):
        df_tmp_res = df_tmp[(df_tmp['是否为宽带客户'] == self.res[0])]
        return df_tmp_res


清理重复号码函数


In [37]:
def df_clean(df, df_no_need):
    df = df[~df.号码.isin(df_no_need.号码)]
    return df


### 条件进队


### 策略模式+工厂模式


In [38]:
# 定义一个接口
class choose(object):
    # 不实现
    def choose_consumer(self):
        pass


### 工厂实现


我号异宽工厂

In [39]:

class whyk_choose(choose):
    def __init__(self, ob, df_tmp,pdl1):
        self.ob = ob
        self.df_tmp = df_tmp
        self.pdl1=pdl1
    def choose_consumer(self):

        # 增强
        return self.ob.df_whyk(self.df_tmp)




我宽异号工厂

In [40]:
 


class wkyh_choose(choose):
    def __init__(self, ob, df_tmp):
        self.ob = ob
        self.df_tmp = df_tmp

    def choose_consumer(self):
        # 增强
        return self.ob.df_wkyh(self.df_tmp)


全家享工厂

In [41]:



class qjx_choose(choose):
    def __init__(self, ob, df_tmp):
        self.ob = ob
        self.df_tmp = df_tmp

    def choose_consumer(self):
        # 增强
        return self.ob.df_qjx(self.df_tmp)



终端购机工厂

In [42]:



class zdgj_choose(choose):
    def __init__(self, ob, df_tmp):
        self.ob = ob
        self.df_tmp = df_tmp

    def choose_consumer(self):
        # 增强
        return self.ob.df_zdgj(self.df_tmp)

### 简单工厂

已改为策略工厂，为方便理解如何将简单工厂改为策略工厂，故注释保留

In [18]:
# consumer_factory
# easy_factory
# class Consumer_factory:

#     def choose_consumer(type, ob, df_tmp):
#         print(type)

#         if '我号' in type:
#             return whyk_choose(ob, df_tmp).choose_consumer()

#         if '全家享' in type:

#             return qjx_choose(ob, df_tmp).choose_consumer()

#         if '终端购机' in type:
#             return zdgj_choose(ob, df_tmp).choose_consumer()
#         if '我宽' in type:
#             print('我宽')
#             return wkyh_choose(ob, df_tmp).choose_consumer()


### 策略模式

自动选择号码类型

In [43]:
class Consumer_strategy():
    dict_map = {}

    def choose_consumer(self):
        pass

    def register(self):
        pass

#各策略
#我号异宽
class whyk_strategy(Consumer_strategy):
    def __init__(self):
        self.register()

    def choose_consumer(self,ob,df_tmp): 
        return whyk_choose(ob, df_tmp).choose_consumer()

    def register(self):
        super().dict_map['我号异宽'] = self

#全家享2.0
class qjx_strategy(Consumer_strategy):
    def __init__(self):
        self.register()

    def choose_consumer(self,ob,df_tmp):
        return qjx_choose(ob, df_tmp).choose_consumer()

    def register(self):
        super().dict_map['全家享2.0'] = self

#终端购机
class zdgj_strategy(Consumer_strategy):
    def __init__(self):
        self.register()

    def choose_consumer(self,ob,df_tmp):
        return zdgj_choose(ob, df_tmp).choose_consumer()

    def register(self):
        super().dict_map['终端购机'] = self

#我宽异号
class wkyh_strategy(Consumer_strategy):
    def __init__(self):
        self.register()

    def choose_consumer(self,ob,df_tmp):
        return wkyh_choose(ob, df_tmp).choose_consumer()

    def register(self):
        super().dict_map['我宽异号'] = self

#上下文Context
class ConsumerContext():
    def __init__(self):
        self.strategy1 = None

    def consumerText(self, consumerStrategy):
        self = consumerStrategy
        return self

    def choose_consumer(self):
        return self.choose_consumer()

#自动决策
class autoType(Consumer_strategy):
    #在初始化手工注入bean，实际上应该引入IOC控制反转，但是太麻烦了，小项目不至于
    def __init__(self):
        self.dict_map = super().dict_map
        #TODO
        whyk_strategy()
        qjx_strategy()
        zdgj_strategy()
        wkyh_strategy()
    def strategy(self, stype):
        return self.dict_map.get(stype)

    条件数字转对应二进制码


基站打标函数(我号异宽)

In [44]:
def station_handle(df):
    df_pivot = df[['号码', '基站ID']].groupby(['基站ID']).count()
    df_pivot.rename(columns={'号码': '数量'}, inplace=True)
    df_res = pd.merge(df, df_pivot, on='基站ID')
    return df_res


In [45]:
# 若移位超出修改format高位补0格式
def get_kh(df_tmp, consumer_type, type, term):
    # consumer_type为客户类型
    df_stack = []
    for i in term:
        i = i << consumer_type  # 移动几位
        res = '{:07b}'.format(i)
        ob = wkkh_term(res)
        # ob=wkkh_term(a,b,c,d,e,f,g)

        #挑选策略

        strategy1 = autoType().strategy(type)       
        # 策略工厂
        context1 = ConsumerContext().consumerText(strategy1)     

        df_res = context1.choose_consumer(ob,df_tmp)

        
        # df_res=choose_consumer(type,ob,df_tmp)#TODO

        df_stack.append(df_res)
    # 结束条件
    df_end = pd.Series(range(0, 10000))
    df_stack.append(df_end)
    return df_stack


打标基站个数函数

分配函数


In [76]:
def divide(df_rule, out_path, r_id, type, df_origin, consumer_dict):
    global df1
    global df2
    global df3
    global df4
    df_res_total = pd.DataFrame()
    df_res_total_l = pd.DataFrame()

    df_stack = []

    consumer_type = 0
    for key in consumer_dict:
        if key in type:
            consumer_type = consumer_dict[key]
            term = term_dict[key]
            break
    # consumer_type移位所需位数,type判断获得哪类类型客户号码
    # term_now=choose_term(type)
    print(consumer_type)

    if '我号' in type:
        #TODO
        for index, row in df_rule.iterrows():
            # 取出各条件号码，依次遍历
            df_res = pd.DataFrame()
            #先选归属
            df_tmp = df_origin[df_origin[r_id] == row[r_id]]
            df_tmp=station_handle(df_tmp)
            pdl1 = row[type]  # 修改条件,派单量
            df_tmp=df_tmp.sort_values(by=['基站成员数','条件'],ascending=False)
            if pdl1<=len(df_tmp):
                df_res=df_tmp.head(pdl1)
            else:
                df_res=df_tmp.head(len(df_tmp))
            df_res_total_l = pd.concat([df_res_total_l, df_res])
    else:
        for index, row in df_rule.iterrows():
            # 取出各条件号码，依次遍历
            df_res = pd.DataFrame()
            #先选归属
            df_tmp = df_origin[df_origin[r_id] == row[r_id]]

            pdl1 = row[type]  # 修改条件,派单量
            df_stack = []


            #通过工厂生产号码
            df_stack = get_kh(df_tmp, consumer_type, type, term)

            # print(len(df_end))
            # 从2开始，有n个条件，就压栈n-1个条件和df_end条件
            pdl1 = int(pdl1)
            row = row['派单编号']
            # 压出队头，利用队列思想
            df_t_tmp = df_stack.pop(0)

            # 递归
            df_res = handle(df_t_tmp, pdl1, row, df_stack,
                            df_res_total, df_res, df_origin)
            df_res_total_l = pd.concat([df_res_total_l, df_res])
            df_origin = df_clean(df_origin, df_res_total_l)

    

    df_res_total_l.to_excel(out_path)
    df4 = df4[~df4.号码.isin(df_res_total_l.号码)]
    df3 = df3[~df3.号码.isin(df_res_total_l.号码)]
    df2 = df2[~df2.号码.isin(df_res_total_l.号码)]
    df1 = df1[~df1.号码.isin(df_res_total_l.号码)]


### 开始分配


In [52]:

term_dict = {'全家享2.0': qjx_term, '我号异宽': whyk_term,
             '我宽异号': wkyh_term, '终端购机': zdgj_term}
consumer_dict = dict({'全家享2.0': 2, '我号异宽': 0, '我宽异号': 2, '终端购机': 6})  # 条件


### 装维


全家享


In [55]:
outpath = '装维-全家享客户-结果.xlsx'
divide(df_r4, outpath, '服务中心ID', '全家享2.0', df3, consumer_dict)


2


### 渠道


我号异宽


In [82]:
outpath = '渠道-我号异宽客户-结果.xlsx'
divide(df_r1, outpath, '网格ID', '我号异宽', df4, consumer_dict)


0


我宽异号


In [ ]:
outpath = '渠道-我宽异号-结果.xlsx'
divide(df_r1, outpath, '网格ID', '我宽异号', df2, consumer_dict)


### 直销


我号异宽


In [ ]:
outpath = '直销-我号异宽-结果.xlsx'
divide(df_r2, outpath, '服务中心ID', '我号异宽', df4, consumer_dict)


我宽异号


In [ ]:
outpath = '直销-我宽异号-结果.xlsx'
divide(df_r2, outpath, '服务中心ID', '我宽异号', df2, consumer_dict)


2


### 三大家


终端购机


In [ ]:
outpath = '三大家-终端购机-结果.xlsx'
divide(df_r3, outpath, '服务中心ID', '终端购机', df1, consumer_dict)


6
